### < Topic >

1. review
    
    1.1 영화 한 편에 대한 crawling과 전처리


2. topic
    
    2.1 단어 분리 (Byte Pair Encoding, BPE) 
    
    2.2 데이터 분리
     

# 단어 분리 (Byte Pair Encoding, BPE)
- Integer Encoding: 단어 별로 빈도수를 체크해서 순위를 매긴 후 댠어를 정수로 표현하는 Encoding
- One-Hot Encoding: 0 / 1로 단어를 표현하는 Encoding, vector화해서 처리

    ⇒ OOV(Out of Vocabulary), UNK(Unknown Token) 문제: 인코딩 결과에 포함되지 않는 단어들
 
 
    - Integer Encoding: 전체 100개 단어 중 빈도수 높은 30개 단어만 사용
    - One-Hot Encoding: vector 차원을 30으로 결정
    - 빈도수가 높은 30개 단어가 아닌 단어들은 인식 불가
    
    
    ⇒ BPE: 모르는 단어가 주어졌을 때 처리하는 방법 (글자 단위로 인코딩)

글자(character) 단위에서 점차적으로 단어 집합(vocabulary)을 만들어내는 Bottom-up 방식

훈련 데이터에 있는 단어들을 모든 글자(character) 또는 유니코드(unicode) 단위로 단어 집합을 만들고, 가장 많이 등장하는 유니그램을 하나의 유니그램으로 통합

## BPE 코드 실습

In [2]:
import re, collections

In [3]:
num_merges = 10

In [5]:
vocab = {'l o w </w>' : 5,
        'l o w e r </w' : 2,
        'n e w e s t </w>' : 6,
        'w i d e s t </w>' : 3
        }

- low, lower, newest, widest : 빈도수

- < /w > : 단어의 맨 끝에 붙이는 특수 문자

- 단어 단위가 아닌 글자 단위이기 때문에 공백을 준다.


    ⇒ BPE 코드는 가장 빈도수가 높은 유니그램의 쌍을 하나의 유니그램으로 통합하는 과정을 num_merges회 반복

In [20]:
# 글자별 빈도수 체크 함수

def get_stats(vocab):
    pairs = collections.defaultdict(int)   # key 없이 사용할 수 있는 defaultdictict
    for word, freq in vocab.items():    # dictionary: key: 단어수 - value: 빈도수
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

In [17]:
# 글자 빈도수에 따른 유니그램 작성 (합치기) 함수

def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

In [21]:
# BPE 수행 함수

for i in range(num_merges):
    pairs = get_stats(vocab)
    best = max(pairs, key=pairs.get)
    vocab = merge_vocab(best, vocab)
    print(best)

('e', 's')
('es', 't')
('est', '</w>')
('l', 'o')
('lo', 'w')
('n', 'e')
('ne', 'w')
('new', 'est</w>')
('low', '</w>')
('w', 'i')


In [22]:
print(vocab)

{'low</w>': 5, 'low e r </w': 2, 'newest</w>': 6, 'wi d est</w>': 3}


###  https://arxiv.org/pdf/1508.07909.pdf
BPE 알고리즘 관련 논문

## WPM (Wordpiece Model)
 하나의 단어를 내부 단어(Subword Unit)들로 분리하는 단어 분리 모델

- WPM 아이디어를 제시한 논문
https://static.googleusercontent.com/media/research.google.com/ko//pubs/archive/37842.pdf


- WPM 아이디어를 제시한 논문을 토대로 구글에서 변형하여 번역기에 사용한 Model
https://arxiv.org/pdf/1609.08144.pdf


- WPM은 BPE의 변형된 형태 (톧계의 가능도(우도, likelihood)) 기반)
- 통계에 기반하여 내부단어(subwords)로 띄어쓰기 분리

- Ex) 
      WPM 수행 이전: Jet makers feud over seat width with big orders at stake
      WPM 수행 결과(wordpieces): _J et _makers _fe ud _over _seat _width _with _big _orders _at _stake

## Google의 Sentencepiece

논문: https://arxiv.org/pdf/1808.06226.pdf
        
Sentencepiece github: https://github.com/google/sentencepiece
    
이점: 단어 분리 알고리즘을 사용하기 위해, 데이터에 단어 토큰화를 먼저 진행한 상태여야 하므로 모든 언어에서 사용하는데는 어려움이 있으나, Sentenpiece는 사전 토큰화 작업 없이 단어 분리 토큰화를 수행하는 알고리즘으로 특정 언어에 종속되지 않는 알고리즘

# 데이터 분리를 위한 Python code 실습

### 1) zip 함수 이용하여 분리하기

- 벡터 차원이 동일해야 함

In [28]:
x, y = zip(['a', 1], ['b', 2], ['c', 3])

print(x)
print(y)

('a', 'b', 'c')
(1, 2, 3)


In [30]:
# Matrix 형식

sequences = [['a', 1], ['b', 2], ['c', 3]]
x, y = zip(*sequences)  # * 추가, 데이터 전체를 unpacking

print(x)
print(y)

('a', 'b', 'c')
(1, 2, 3)


### 2) 데이터프레임 이용하여 분리하기

In [34]:
import pandas as pd

values = [['당신에게 드리는 마지막 혜택!', 1],
['내일 뵐 수 있을지 확인 부탁드...', 0],
['도연씨. 잘 지내시죠? 오랜만입...', 0],
['(광고) AI로 주가를 예측할 수 있다!', 1]]
columns = ['메일 본문', '스팸 메일 유무']

df = pd.DataFrame(values, columns = columns) ; df

,메일 본문,스팸 메일 유무
0,당신에게 드리는 마지막 혜택!,1
1,내일 뵐 수 있을지 확인 부탁드...,0
2,도연씨. 잘 지내시죠? 오랜만입...,0
3,(광고) AI로 주가를 예측할 수 있다!,1


In [35]:
x = df['메일 본문']
y = df['스팸 메일 유무']

In [36]:
print(y)

0    1
1    0
2    0
3    1
Name: 스팸 메일 유무, dtype: int64


### 3) Numpy 이용하여 분리하기

In [37]:
import numpy as np
ar = np.arange(0, 16).reshape((4,4)) ; ar

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

In [38]:
[앞: 데이터 범위, 뒤: 행렬]

x = ar[:, :3] ; x  # 전체 데이터 중 3열까지

array([[ 0,  1,  2],
       [ 4,  5,  6],
       [ 8,  9, 10],
       [12, 13, 14]])

In [39]:
y = ar[:, 3] ; y   # 전체 데이터 중 4열만

array([ 3,  7, 11, 15])

## 테스트 데이터 분리하기

### Scikit-learn 이용하여 분리하기

In [41]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, t_test = train_test_split(x, y, test_size = 0.2, random_state = 1234)

In [42]:
x, y = np.arange(10).reshape((5,2)), range(5)

print(x)
print(list(y))

[[0 1]
 [2 3]
 [4 5]
 [6 7]
 [8 9]]
[0, 1, 2, 3, 4]


In [46]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.33, random_state = 123)

print(x_train)
print(x_test)

[[8 9]
 [0 1]
 [4 5]]
[[2 3]
 [6 7]]


In [47]:
print(y_train)
print(y_test)

[4, 0, 2]
[1, 3]
